In [1]:
import numpy as np
import pandas
import pm4py
from matplotlib import pyplot as plt
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
import ot
import os
from tqdm import tqdm
import collections
import itertools
import pickle
import matplotlib.dates as md


from pix_framework.statistics.distribution import DurationDistribution, get_best_fitting_distribution, DistributionType

import sys
sys.path.append('../../TaskExecutionTimeMining/')
from drbart_parser import *
from event_log_transformer import *

sys.path.append('../../Evaluation/')
from process_evaluation import *

In [2]:
file_path = '../../../data/BPI Challenge 2017.xes'
event_log = pm4py.read_xes(file_path)

parsing log, completed traces ::   0%|          | 0/31509 [00:00<?, ?it/s]

In [3]:
event_log = TransformEventLog.seconds_in_day(event_log)

In [6]:
from dumas_model import DumasModel

dm = DumasModel(event_log)


with open('../../../models/bpic_2017_dumas_model/dumas_model.pickle', 'wb') as handle:
    pickle.dump(dm, handle)


/home/michel/Documents/TaskExecutionTimeMining/src/notebooks/BPIC_2017/../../TaskExecutionTimeMining/event_log_transformer.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_end_event_log.loc[:, 'duration'] = start_end_event_log[timestamp_name + complete_name_gen] - start_end_event_log[timestamp_name + start_name_gen]
/home/michel/Documents/TaskExecutionTimeMining/src/notebooks/BPIC_2017/../../TaskExecutionTimeMining/event_log_transformer.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_en

In [5]:
with open('../../../models/bpic_2017_dumas_model/dumas_model.pickle', 'rb') as handle:
    dm = pickle.load(handle)

EOFError: Ran out of input

In [ ]:
import importlib
importlib.reload(sys.modules['process_evaluation'])
from process_evaluation import *

dumas_model = SampleOutcomes_Dumas_BPIC2017(event_log, dm)


In [ ]:
from pm4py.objects.petri_net import semantics
from scipy.stats import gaussian_kde


get_real_end_time = lambda case_name : event_log[(event_log['case:concept:name'] == case_name)]['time:timestamp'].max().timestamp()
results = []
n = 1000
i = 0
for case_name in event_log['case:concept:name'].unique():
    i += 1
    if i < 2:
        continue
    case_samples = [dumas_model.sample_case(case_name) for i in range(n)]
    real_end_time = get_real_end_time(case_name)

    likelihood = kde_likelihood(case_samples, real_end_time) * (3600*24)
    
    case_samples = [datetime.datetime.fromtimestamp(ts) for ts in case_samples]
    real_end_time = datetime.datetime.fromtimestamp(real_end_time)

    plt.figure(figsize=(14, 6))
    #plt.xticks( rotation=25 )
    #print(case_samples)
    ax=plt.gca()
    xfmt = md.DateFormatter('%Y-%m-%d')
    ax.xaxis.set_major_formatter(xfmt)
    ax.xaxis.set_major_locator(md.WeekdayLocator(byweekday=md.SU, interval=1))
    mxfmt = md.DateFormatter('%a')
    ax.xaxis.set_minor_formatter(mxfmt)
    ax.xaxis.set_minor_locator(md.DayLocator())
    ax.tick_params(axis='both', which='major', labelsize=8, rotation=90)
    ax.tick_params(axis='both', which='minor', labelsize=5, rotation=90)
    
    plt.gcf().autofmt_xdate()
    plt.gcf().set_dpi(300)
    plt.hist(case_samples, bins=pandas.date_range(start= pandas.to_datetime(min(case_samples)).floor('D'), end=pandas.to_datetime(max(case_samples)).ceil('D'), freq='6H'), density=True)
    plt.axvline(x=real_end_time, color='red')
    plt.show()
    results.append(likelihood[0])
    print(case_name, likelihood, min(case_samples), max(case_samples),  max(case_samples)-min(case_samples))

In [ ]:
results

In [ ]:
ll = np.log(np.array(results))
ll

In [ ]:
ll[:28]
np.median(ll[:28])

In [ ]:
np.mean(np.array(results[:28]))